In [174]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from dataclasses import dataclass
from typing import Union
import re
import time
import pickle
from selenium.webdriver import ActionChains
from selenium.webdriver.remote.errorhandler import ElementNotInteractableException

# https://www.browserstack.com/guide/how-to-handle-captcha-in-selenium

In [175]:
url = "https://www.1001tracklists.com/tracklist/2q5y06k9/alesso-mainstage-creamfields-chile-2024-11-17.html"
chop = webdriver.ChromeOptions()
chop.add_extension("/Users/johncabrahams/Downloads/gighmmpiobklfepjocnamgkkbiglidom.crx")
driver = webdriver.Chrome(options = chop)
time.sleep(5)
chld = driver.window_handles[1]
driver.switch_to.window(chld)
driver.close()
current_tab=driver.window_handles[0]
driver.switch_to.window(current_tab)
driver.get(url)
# time.sleep(5)

set = {}

In [176]:
def _get_id(atom):
    match = re.search(r"(trRow)\d+", atom.get_attribute("class"))
    return int(re.search(r"\d+", match.group()).group())

In [177]:
"""This method gets the date of publication, the number of tracks, and the name of the mix."""
# Get all the meta data elements for a set
moduleMetaDataElements = driver.find_elements("xpath", "//body[@id='body']//meta")
# Relevant and irrelevant meta data names
relevantMetaDataTypes = ["name", "datePublished", "numTracks", "genre"]
irrelevantMetaDataValues = ['tracklist', '1001Tracklists', 'YouTube video']
# Sort through only the relevant information in `moduleMetaDataElements`
moduleMetaDataRelevantInfo = [metadata for metadata in moduleMetaDataElements if metadata.get_attribute("itemprop") in relevantMetaDataTypes and metadata.get_attribute("content") not in irrelevantMetaDataValues]
moduleMetaData = [(metadata.get_attribute("itemprop"), metadata.get_attribute("content")) for metadata in moduleMetaDataRelevantInfo][0:5]
# Create a dictionary of values from the metadata
moduleMetaDataDict = {k : v for k, v in moduleMetaData}
# Save the relevant values found
name = moduleMetaDataDict['name']
date_published = moduleMetaDataDict['datePublished']
num_tracks = moduleMetaDataDict['numTracks']

In [179]:
driver.find_elements(By.CLASS_NAME, "trRow1")

[<selenium.webdriver.remote.webelement.WebElement (session="2108d5aa41a0196bd6383d2f933b7bc2", element="f.BAEC24BB5BE9E397654DCC4A658BBA5B.d.5F9F371FA69C485EFAB58F6DF030DF06.e.358")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2108d5aa41a0196bd6383d2f933b7bc2", element="f.BAEC24BB5BE9E397654DCC4A658BBA5B.d.5F9F371FA69C485EFAB58F6DF030DF06.e.359")>]

The following retrieves the data pertaining to the songs.

In [168]:
def getVisibleSongLink(song_atom):
    link_classes = ["fa-soundcloud", "fa-youtube-play", "fa-spotify", "fa-apple"]
        
    ActionChains(driver).scroll_by_amount(0, -300).perform()
    time.sleep(2)
    ActionChains(driver).scroll_to_element(song_atom).perform()
    time.sleep(2)
    ActionChains(driver).scroll_by_amount(0, 300).perform()
    time.sleep(2)
    links = [song_atom.find_elements(By.CLASS_NAME, link) for link in link_classes]
    links_reduced = [link for link in links if link != []]
    if links_reduced == []:
        return
    links_reduced = [link[0] for link in links_reduced]
    print(str(song_atom.find_element(By.TAG_NAME, "span").text))
    print(len(links_reduced))
    all_actual_links = []

    for link in links_reduced:
        try:
            link.click()
        except ElementNotInteractableException:
            continue
        time.sleep(2)
            
        if not link.get_attribute("onclick"):
            continue
        link_window = driver.find_element(By.CLASS_NAME, "mP") # f border rB")
        link_close_button = link_window.find_element(By.CLASS_NAME, "close")
        actual_link = link_window.find_element(By.TAG_NAME, 'iframe').get_attribute("src")
        all_actual_links.append(actual_link)
        link_close_button.click()
        time.sleep(2)
    return all_actual_links

calculate the ratio of scroll so 300 to the total height of the page we have right now...

https://stackoverflow.com/questions/77653286/how-to-download-songs-from-soundcloud-theyre-no-longer-providing-api-keys

In [169]:
"""This method gets the songs of the set"""
# This expression gets the visible atoms. 
visible_song_atoms = driver.find_elements(By.CLASS_NAME, "tlpTog")
# This expression gets the invisible atoms (i.e., those binded to a mix with an ellipses (...))
invisible_song_atoms = driver.find_elements(By.CLASS_NAME, "tlpSubTog")
        
# Loop through the visible song atoms and get their corresponding metadata
for i in range(len(visible_song_atoms)):
    
    # 1. Get the visible song's data attributes, including the song link
    main_song_atom = song_atom = visible_song_atoms[i]
    currentIndex = song_atom.find_element(By.TAG_NAME, "span").text
    metadata = song_atom.find_elements(By.TAG_NAME, "meta")
    play_time = song_atom.find_elements(By.CLASS_NAME, "cue")[0].text
    retrievedMetaData = {}
    for data in metadata:
        retrievedMetaData[data.get_attribute("itemprop")] = data.get_attribute("content")
    set[_get_id(song_atom)] = {"currentIndex": currentIndex, "metadata": retrievedMetaData, "time": play_time, "links": getVisibleSongLink(song_atom)}

    # 2. Check for any mashups
    # Hidden songs are indicated by the tgBtn (toggle button) class name within the song atom
    if len(main_song_atom.find_elements(By.CLASS_NAME, "tgBtn")) == 1:
        invisible_atoms = []
        # Loop through the invisible atoms parsed on our initial scan (they show up on the html code DOM, but are hidden from the user)
        for invisible_atom in invisible_song_atoms:
            invisible_atom_id = _get_id(invisible_atom)
            if invisible_atom_id == _get_id(main_song_atom):
                invisible_atoms.append(invisible_atom)
        # Click the button to make the invisible song atoms visible
        main_song_atom.find_element(By.CLASS_NAME, "tgBtn").click()
        for invisible_atom in invisible_atoms:
            currentIndex = invisible_atom.find_element(By.TAG_NAME, "span").text
            metadata = invisible_atom.find_elements(By.TAG_NAME, "meta")
            id = _get_id(invisible_atom)
            retrievedMetaData = {}
            for data in metadata:
                retrievedMetaData[data.get_attribute("itemprop")] = data.get_attribute("content")
            set[id]['invisible_atoms'] = {"currentIndex": currentIndex, "metadata":retrievedMetaData, "links":getVisibleSongLink(invisible_atom)}
        # Scroll up to the atom that contained the invisible mashups
        time.sleep(3)
        ActionChains(driver).scroll_to_element(main_song_atom).perform()
        ActionChains(driver).scroll_by_amount(0, -300).perform()
        # ActionChains(driver).scroll_to_element(song_atom).perform()
        time.sleep(3)
        # Click the button to make the visible song atoms invisible
        main_song_atom.find_element(By.CLASS_NAME, "tgBtn").click()
        ActionChains(driver).scroll_by_amount(0, 300).perform()

    # 3. Check for any binded (mixed with) songs, indicated by a 'w/'
    try:
        # This will get the elements that are not invisible but bind to a previous song with a `w/` indicator.
        while visible_song_atoms[i+1].find_element(By.TAG_NAME, "span").text.rstrip().lstrip() == 'w/':
            print("with found")
            i += 1
            song_atom = visible_song_atoms[i]
            currentIndex = song_atom.find_element(By.TAG_NAME, "span").text
            metadata = song_atom.find_elements(By.TAG_NAME, "meta")
            play_time = song_atom.find_elements(By.CLASS_NAME, "cue")[0].text
            retrievedMetaData = {}
            for data in metadata:
                retrievedMetaData[data.get_attribute("itemprop")] = data.get_attribute("content")
            id = _get_id(song_atom)
            set[id]['binded_atoms'] = {"currentIndex": currentIndex, "metadata": retrievedMetaData, "time": play_time}
            set[id]["links"] = getVisibleSongLink(song_atom)   
            print("with done")   
    # An IndexError means we indexed outside of the appropriate range, and therefore must be at the end of the loop.
    except IndexError:
        break

01
3
with found
w/
3
with done
w/
3


KeyboardInterrupt: 

In [170]:
set

{1: {'currentIndex': '01',
  'metadata': {'name': 'Alesso & SICK INDIVIDUALS & Sentinel - Upside Down',
   'byArtist': 'Alesso & SICK INDIVIDUALS & Sentinel',
   'publisher': '<span class="blueTxt"><span title="label" class="trackLabel noWrap"><a href="/label/7d3x3yz/tomorrowland-music/index.html" translate="no" class="notranslate " title="open label page">Tomorrowland Music</a></span></span>',
   'duration': 'PT3M27S',
   'url': '/track/d83ljwcp/alesso-sick-individuals-sentinel-upside-down/index.html'},
  'time': '',
  'links': ['https://w.soundcloud.com/player/?url=https://api.soundcloud.com/tracks/111117742&auto_play=true&show_artwork=true&color=%23ff5500&hide_related=true&show_comments=false&show_user=true&show_reposts=false&show_teaser=false&visual=true',
   'https://open.spotify.com/embed/track/3IRopjEQzdRC8oP98yUikY?theme=0',
   'https://embed.music.apple.com/us/album/under-control-feat-hurts-extended-mix/772057010?i=772057017app=music&at=1000lwkw?'],
  'binded_atoms': {'current

The following cleans up our instance

In [159]:
saved_full_set = set

In [172]:
file = "_".join([name, date_published]) + ".mx"
with open(file, 'wb') as file:
    pickle.dump(saved_full_set, file)
driver.quit()

In [181]:
import random
import string
from datetime import datetime, timedelta
from dataclasses import dataclass

@dataclass
class AtomicTrack:
    index: str
    name: str
    artist: str
    publisher: str
    url: str
    time: str
    links: list[str]

@dataclass
class CompoundTrack:
    index: str
    name: str
    artist: str
    publisher: str
    url: str
    time: str
    links: list[str]
    binded_atoms: list[AtomicTrack]
    invisible_atoms: list[AtomicTrack]

@dataclass
class Set:
    name: str
    date_published: str
    num_tracks: str
    url_link: str
    tracks: list[CompoundTrack]

In [ ]:
# NB: ChatGPT Code!!

def random_string(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def random_url():
    return f"https://example.com/{random_string(10)}"

def random_time():
    return f"{random.randint(0, 5)}:{random.randint(0, 59):02d}"  # Random duration in minutes:seconds

def generate_atomic_track(index):
    return AtomicTrack(
        index=f"AT-{index}",
        name=f"AtomicTrack-{random_string(6)}",
        artist=f"Artist-{random_string(5)}",
        publisher=f"Publisher-{random_string(4)}",
        url=random_url(),
        time=random_time(),
        links=[random_url() for _ in range(random.randint(1, 3))]
    )

def generate_compound_track(index):
    binded_atoms = [generate_atomic_track(i) for i in range(random.randint(1, 3))]
    invisible_atoms = [generate_atomic_track(i + 10) for i in range(random.randint(0, 2))]
    return CompoundTrack(
        index=f"CT-{index}",
        name=f"CompoundTrack-{random_string(6)}",
        artist=f"Artist-{random_string(5)}",
        publisher=f"Publisher-{random_string(4)}",
        url=random_url(),
        time=random_time(),
        links=[random_url() for _ in range(random.randint(1, 3))],
        binded_atoms=binded_atoms,
        invisible_atoms=invisible_atoms
    )

def generate_set(name, num_tracks):
    date_published = (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")
    url_link = random_url()
    tracks = [generate_compound_track(i) for i in range(num_tracks)]
    return Set(name, date_published, str(num_tracks), url_link, tracks)

# Generate a synthetic dataset with 5 songs
synthetic_set = generate_set("Synthwave Collection", 5)

# Example output
print(synthetic_set)

Set(name='Synthwave Collection', date_published='2025-02-04', num_tracks='5', url_link='https://example.com/iXfaenOUij', tracks=[CompoundTrack(index='CT-0', name='CompoundTrack-mgtJ1L', artist='Artist-XFcvL', publisher='Publisher-9sFh', url='https://example.com/VSYLCeaNiP', time='3:56', links=['https://example.com/R0M295pAND', 'https://example.com/4Evn1HaFrk'], binded_atoms=[AtomicTrack(index='AT-0', name='AtomicTrack-LwFwQm', artist='Artist-21h31', publisher='Publisher-W4xx', url='https://example.com/arBOpcKJpt', time='0:55', links=['https://example.com/YK59mOeelu', 'https://example.com/2HhYkrhtQi', 'https://example.com/n7F2J03mW6']), AtomicTrack(index='AT-1', name='AtomicTrack-cneI92', artist='Artist-s19Ju', publisher='Publisher-t74h', url='https://example.com/ucnmG8KlC5', time='3:51', links=['https://example.com/IIEbbLYbD2', 'https://example.com/Zpr2Ba97WP', 'https://example.com/AtrYCAxm9t']), AtomicTrack(index='AT-2', name='AtomicTrack-m70LVL', artist='Artist-5Dwnf', publisher='Pub

In [ ]:
from graphviz import Digraph


def trace(compound_song):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v.binded_atoms:
                edges.add((child, v))
                build(child)
            for child in v.invisible_atoms:
                edges.add((child, v))
                build(child)
    build(compound_song)
    return nodes, edges

def draw_dot(compound_song):
    dot = Digraph(format='svg', graph_attr={'rankdir':'LR'}) # LR = left to right

    nodes, edges = trace(compound_song)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label= "{name %s | index %s | time %s}" % (n.name, n.index, n.time), shape='record')

        if len(n.binded_atoms) != 0:
            # if this value ios a result of some operation, create an op node for it
            dot.node(name=uid + "binded with", label = "binded with")
            # and connect this node to it
            dot.edge(uid + "binded with", uid)
        if len(n.invisible_atoms) != 0:
            # if this value ios a result of some operation, create an op node for it
            dot.node(name=uid + "mashup with", label = "mashup with")
            # and connect this node to it
            dot.edge(uid + "mashup with", uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    return dot